In [1]:
import imagine as im
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg

2025-10-03 12:39:39.584 | INFO     | imagine.config:<module>:11 - PROJ_ROOT path is: /Users/saurabh.ranjan/Library/CloudStorage/OneDrive-UniversityofFlorida/2020_2025_reality_monitoring_dissertation/E/osf-imagine/imagine_world_model


In [2]:
data_path = im.config.PROCESSED_DATA_DIR

In [3]:
def get_data_stats(measure,task,stats_files_prefix=None,columns = None):
    if stats_files_prefix is None:
        stats_file_prefix = "assumption_checks"
    
    if columns is None:
        columns = ['X', 'Y', 'pearson_r', 'pearson_p_unc', 'pearson_p_corr',
         'spearman_rho', 'spearman_p_unc', 'spearman_p_corr', 'shapiro_x_p','shapiro_y_p']
    df = pd.read_csv(data_path/f"{stats_file_prefix}_{task}_{measure}.csv")
    
    df["measure"] = measure
    df["task"] = task
    df_reduced = df.loc[:,columns+["task","measure"]]

    return df_reduced




vviq_measures = {}
psiq_measures = {}
vviq = "vv"
psiq = "ps"
imagination_task = ["vv", "ps" ] # imagine environemnt and imagine sensory
measures = ['expectedinfluence', 'strength','closeness','betweenness']
for task in imagination_task:
    for measure in measures:
        if task == vviq:
            vviq_measures[f"{measure}"] = get_data_stats(measure,task)
        elif task == psiq:
            psiq_measures[f"{measure}"] = get_data_stats(measure,task)

all_vviq_measures = pd.concat(vviq_measures.values(),ignore_index=True)
all_psiq_measures = pd.concat(psiq_measures.values(),ignore_index=True)
            
all_measues = pd.concat([all_vviq_measures,all_psiq_measures],ignore_index=True)



In [4]:
all_vviq_measures["X"].unique()
humans = ['uk_human', 'ukfl_human','fl_human', 'pl1_human', 'pl2_human', 'pl_human', 'plfl_human']
ai_models = ['independent_g12', 'independent_g12qt', 'independent_g27',
       'independent_g27qt', 'independent_ll70', 'independent_llsc',
       'cumulative_g12', 'cumulative_g12qt', 'cumulative_g27',
       'cumulative_g27qt', 'cumulative_ll70','cumulative_llsc']

all_vviq_measures["X_human"] = all_vviq_measures["X"].apply(lambda x: x in humans)
all_vviq_measures["Y_human"] = all_vviq_measures["Y"].apply(lambda x: x in humans)

all_psiq_measures["X_human"] = all_psiq_measures["X"].apply(lambda x: x in humans)
all_psiq_measures["Y_human"] = all_psiq_measures["Y"].apply(lambda x: x in humans)


In [5]:
comparison_groups = ["human-human","human-ai","ai-ai"]

comparison_groups_lambda = lambda row: "human-human" if row["X_human"] and row["Y_human"] else ("ai-ai" if not row["X_human"] and not row["Y_human"] else "human-ai")

all_vviq_measures["comparison_group"] = all_vviq_measures.apply(comparison_groups_lambda,axis=1)
all_psiq_measures["comparison_group"] = all_psiq_measures.apply(comparison_groups_lambda,axis=1)

In [6]:

def highlight_column_significant(s,threshold=0.5):
    # Example: Highlight values greater than 50 in green, otherwise no style
    return ['background-color: lightgreen' if v < threshold else '' for v in s]

def highlight_rows_significant(row,threshold=0.05):
    return ['background-color: lightgreen' if v < threshold else '' for v in row]


def highlight_rows(row,threshold=0.05):
    # Assuming 'Score' is the column to check and we want to highlight if Score > 80
    color = ""
    if row['pearson_p_unc'] < threshold:
        color = "lime"
        if (row['shapiro_x_p'] < threshold) | (row['shapiro_y_p'] < threshold):
            color = "darkseagreen"
    

    if row['pearson_p_corr'] < threshold:
        color = "skyblue"
        if (row['shapiro_x_p'] < threshold) | (row['shapiro_y_p'] < threshold):
            color = "teal"

    return [f'background-color: {color}'] * len(row)


In [7]:
# human_human 
all_vviq_measures_human = all_vviq_measures.loc[all_vviq_measures["comparison_group"]=="human-human"]
# Move 'colB' to index 0 (first position)
col_to_move = all_vviq_measures_human.pop('measure')  # Remove the column and store its data
all_vviq_measures_human.insert(0, 'measure', col_to_move)
# Apply styling to 'col1'
thres_puncorr_df_hh = all_vviq_measures_human.style.apply(highlight_rows_significant, subset=["pearson_p_unc"])
thres_pcorr_df_hh = all_vviq_measures_human.style.apply(highlight_rows_significant, subset=["pearson_p_corr"])
threh_p_df_hh = all_vviq_measures_human.style.apply(highlight_rows,axis=1)


In [8]:
# When P value uncorrected significant
# when pearson assumption violations: darkseagreen
# when no assumption violations: lime

# When P value corrected significant
# when pearson assumption violations: teal
# when no assumption violations: skyblue

threh_p_df_hh

,measure,X,Y,pearson_r,pearson_p_unc,pearson_p_corr,spearman_rho,spearman_p_unc,spearman_p_corr,shapiro_x_p,shapiro_y_p,task,X_human,Y_human,comparison_group
0,expectedinfluence,fl_human,pl1_human,0.589003,0.000195,0.002413,0.580645,0.000247,0.003730,0.948241,0.668476,vv,True,True,human-human
1,expectedinfluence,fl_human,pl2_human,0.465904,0.003600,0.030600,0.417522,0.008712,0.057843,0.948241,0.673937,vv,True,True,human-human
2,expectedinfluence,fl_human,pl_human,0.595242,0.000163,0.002217,0.574413,0.000293,0.003984,0.948241,0.571948,vv,True,True,human-human
3,expectedinfluence,fl_human,plfl_human,0.732278,0.000001,0.000022,0.725440,0.000001,0.000030,0.948241,0.523667,vv,True,True,human-human
16,expectedinfluence,pl1_human,pl2_human,0.711779,0.000002,0.000048,0.709311,0.000003,0.000053,0.668476,0.673937,vv,True,True,human-human
17,expectedinfluence,pl1_human,pl_human,0.930937,0.000000,0.000000,0.935850,0.000000,0.000000,0.668476,0.571948,vv,True,True,human-human
18,expectedinfluence,pl1_human,plfl_human,0.920320,0.000000,0.000000,0.896628,0.000000,0.000000,0.668476,0.523667,vv,True,True,human-human
31,expectedinfluence,pl2_human,pl_human,0.860134,0.000000,0.000000,0.789223,0.000000,0.000001,0.673937,0.571948,vv,True,True,human-human
32,expectedinfluence,pl2_human,plfl_human,0.837148,0.000000,0.000000,0.757698,0.000000,0.000007,0.673937,0.523667,vv,True,True,human-human
45,expectedinfluence,pl_human,plfl_human,0.970499,0.000000,0.000000,0.943915,0.000000,0.000000,0.571948,0.523667,vv,True,True,human-human


In [9]:
# human_human 
all_psiq_measures_human = all_psiq_measures.loc[all_psiq_measures["comparison_group"]=="human-human"]
# Move 'colB' to index 0 (first position)
col_to_move = all_psiq_measures_human.pop('measure')  # Remove the column and store its data
all_psiq_measures_human.insert(0, 'measure', col_to_move)
# Apply styling to 'col1'
thres_puncorr_df_hh = all_psiq_measures_human.style.apply(highlight_rows_significant, subset=["pearson_p_unc"])
thres_pcorr_df_hh = all_psiq_measures_human.style.apply(highlight_rows_significant, subset=["pearson_p_corr"])
threh_p_df_hh = all_psiq_measures_human.style.apply(highlight_rows,axis=1)


In [10]:
threh_p_df_hh

,measure,X,Y,pearson_r,pearson_p_unc,pearson_p_corr,spearman_rho,spearman_p_unc,spearman_p_corr,shapiro_x_p,shapiro_y_p,task,X_human,Y_human,comparison_group
0,expectedinfluence,fl_human,uk_human,0.425100,0.027364,0.119717,0.315584,0.081727,0.245180,0.867780,0.085923,ps,True,True,human-human
1,expectedinfluence,fl_human,ukfl_human,0.913660,0.000000,0.000000,0.848052,0.000001,0.000063,0.867780,0.037498,ps,True,True,human-human
14,expectedinfluence,uk_human,ukfl_human,0.732608,0.000079,0.004171,0.622078,0.001302,0.034171,0.085923,0.037498,ps,True,True,human-human
105,strength,fl_human,uk_human,0.188723,0.206313,0.528364,0.142857,0.268370,0.575419,0.720861,0.130797,ps,True,True,human-human
106,strength,fl_human,ukfl_human,0.857651,0.000000,0.000035,0.809091,0.000004,0.000470,0.720861,0.095718,ps,True,True,human-human
119,strength,uk_human,ukfl_human,0.538053,0.005935,0.112988,0.450649,0.020173,0.211813,0.130797,0.095718,ps,True,True,human-human
210,closeness,fl_human,uk_human,0.759325,0.000033,0.001147,0.718182,0.000123,0.002582,0.659978,0.228248,ps,True,True,human-human
211,closeness,fl_human,ukfl_human,0.874132,0.000000,0.000012,0.896104,0.000000,0.000002,0.659978,0.297005,ps,True,True,human-human
224,closeness,uk_human,ukfl_human,0.842069,0.000001,0.000044,0.798701,0.000007,0.000372,0.228248,0.297005,ps,True,True,human-human
315,betweenness,fl_human,uk_human,0.385448,0.042210,0.646283,0.460059,0.017932,0.638801,0.536747,0.329842,ps,True,True,human-human


In [11]:
# human ai vviq 
all_vviq_measures_humanai = all_vviq_measures.loc[all_vviq_measures["comparison_group"]=="human-ai"]
# Move 'colB' to index 0 (first position)
col_to_move = all_vviq_measures_humanai.pop('measure')  # Remove the column and store its data
all_vviq_measures_humanai.insert(0, 'measure', col_to_move)
# Apply styling to 'col1'
thres_puncorr_df_ha = all_vviq_measures_humanai.style.apply(highlight_rows_significant, subset=["pearson_p_unc"])
thres_pcorr_df_ha = all_vviq_measures_humanai.style.apply(highlight_rows_significant, subset=["pearson_p_corr"])
threh_p_df_ha = all_vviq_measures_humanai.style.apply(highlight_rows,axis=1)
threh_p_df_ha

,measure,X,Y,pearson_r,pearson_p_unc,pearson_p_corr,spearman_rho,spearman_p_unc,spearman_p_corr,shapiro_x_p,shapiro_y_p,task,X_human,Y_human,comparison_group
4,expectedinfluence,fl_human,independent_g12,0.067870,0.356032,0.538005,0.270528,0.067127,0.198235,0.948241,0.268392,vv,True,False,human-ai
5,expectedinfluence,fl_human,independent_g12qt,-0.204590,0.869328,0.938323,-0.087610,0.683247,0.780854,0.948241,0.026723,vv,True,False,human-ai
6,expectedinfluence,fl_human,independent_g27,0.057012,0.378308,0.541577,0.094208,0.304024,0.464576,0.948241,0.167826,vv,True,False,human-ai
7,expectedinfluence,fl_human,independent_g27qt,0.323383,0.035505,0.146325,0.370235,0.018498,0.093175,0.948241,0.406585,vv,True,False,human-ai
8,expectedinfluence,fl_human,independent_ll70,-0.106802,0.719647,0.822118,-0.057185,0.622050,0.739936,0.948241,0.000026,vv,True,False,human-ai
9,expectedinfluence,fl_human,independent_llsc,0.019599,0.457606,0.616182,0.096408,0.299829,0.464576,0.948241,0.076610,vv,True,False,human-ai
10,expectedinfluence,fl_human,cumulative_g12,0.071263,0.349164,0.533554,0.170455,0.175481,0.336133,0.948241,0.815070,vv,True,False,human-ai
11,expectedinfluence,fl_human,cumulative_g12qt,0.280000,0.060315,0.190763,0.272361,0.065767,0.198235,0.948241,0.000191,vv,True,False,human-ai
12,expectedinfluence,fl_human,cumulative_g27,0.188154,0.151210,0.321321,0.247801,0.085744,0.215947,0.948241,0.124352,vv,True,False,human-ai
13,expectedinfluence,fl_human,cumulative_g27qt,-0.056180,0.619969,0.739612,-0.059751,0.627350,0.739936,0.948241,0.776601,vv,True,False,human-ai


In [12]:
# human_ai psiq 
all_psiq_measures_humanai = all_psiq_measures.loc[all_psiq_measures["comparison_group"]=="human-ai"]
# Move 'colB' to index 0 (first position)
col_to_move = all_psiq_measures_humanai.pop('measure')  # Remove the column and store its data
all_psiq_measures_humanai.insert(0, 'measure', col_to_move)
# Apply styling to 'col1'
thres_puncorr_df_ha = all_psiq_measures_humanai.style.apply(highlight_rows_significant, subset=["pearson_p_unc"])
thres_pcorr_df_ha = all_psiq_measures_humanai.style.apply(highlight_rows_significant, subset=["pearson_p_corr"])
threh_p_df_ha = all_psiq_measures_humanai.style.apply(highlight_rows,axis=1)
threh_p_df_ha

,measure,X,Y,pearson_r,pearson_p_unc,pearson_p_corr,spearman_rho,spearman_p_unc,spearman_p_corr,shapiro_x_p,shapiro_y_p,task,X_human,Y_human,comparison_group
2,expectedinfluence,fl_human,independent_g12,0.432359,0.025147,0.119717,0.296104,0.096238,0.259103,0.867780,0.044713,ps,True,False,human-ai
3,expectedinfluence,fl_human,independent_g12qt,0.387832,0.041177,0.166293,0.289610,0.101439,0.266276,0.867780,0.957297,ps,True,False,human-ai
4,expectedinfluence,fl_human,independent_g27,-0.280576,0.891018,0.895283,-0.275325,0.886468,0.894992,0.867780,0.161392,ps,True,False,human-ai
5,expectedinfluence,fl_human,independent_g27qt,0.283136,0.106808,0.241783,0.310390,0.085438,0.249195,0.867780,0.469398,ps,True,False,human-ai
6,expectedinfluence,fl_human,independent_ll70,0.217575,0.171713,0.323135,0.231169,0.156674,0.329015,0.867780,0.517800,ps,True,False,human-ai
7,expectedinfluence,fl_human,independent_llsc,-0.186690,0.791114,0.834243,-0.166234,0.764289,0.794558,0.867780,0.593144,ps,True,False,human-ai
8,expectedinfluence,fl_human,cumulative_g12,0.283534,0.106472,0.241783,0.325974,0.074642,0.238554,0.867780,0.971688,ps,True,False,human-ai
9,expectedinfluence,fl_human,cumulative_g12qt,0.045891,0.421709,0.546659,0.031169,0.446654,0.581863,0.867780,0.720157,ps,True,False,human-ai
10,expectedinfluence,fl_human,cumulative_g27,0.354014,0.057692,0.182954,0.136364,0.277798,0.422736,0.867780,0.114307,ps,True,False,human-ai
11,expectedinfluence,fl_human,cumulative_g27qt,0.206533,0.184529,0.328399,0.198701,0.193941,0.351100,0.867780,0.382638,ps,True,False,human-ai
